# Supplying Oil Well Sites
This notebook demonstrates the processing of data collected by IoT sensors at multiple oil wells, for the purpose of prioritizing resupply of each well site.  Various chemicals are consumed at each well site at different rates and need to be replenished when running low.  In order to minimize resupply travel and costs, a prioritization algorithm is developed to yield the sites and routing of resupply trucks.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sagemaker import get_execution_role
from  more_itertools import unique_everseen

%matplotlib inline

## Data Discovery

Let's take a look at the data and get a handle on what we're looking at.

In [ ]:
raw_data = pd.read_csv('onica3.csv')    

Look at the first few rows of data

In [ ]:
raw_data.head(20)

Different rows appear to contain different information and populate different columns.  We're going to have to clean this up before we can do much with it.  Let's check how many different kinds of rows there are:

In [ ]:
list(unique_everseen(raw_data['point']))

How many sites and assets (chemical types):

In [ ]:
len(list(unique_everseen(raw_data['site'])))

In [ ]:
len(list(unique_everseen(raw_data['asset'])))

In [ ]:
len(list(unique_everseen(zip(raw_data['site'],raw_data['asset']))))

## ETL (Extract, Transform and Load)
The times are in the file as Unix epoch time (here, milliseconds from 1970-01-01); we'll turn them into datetime objects for clarity.

In [ ]:
raw_data['rollupStartTimestampMs'] = pd.to_datetime(raw_data['rollupStartTimestampMs'], unit='ms')
raw_data['rollupEndTimestampMs'] = pd.to_datetime(raw_data['rollupEndTimestampMs'], unit='ms')
raw_data['latestObservationTimestamp'] = pd.to_datetime(raw_data['latestObservationTimestamp'], unit='ms')

In [ ]:
raw_data.head(20)

Some of the data rows are attributes of the site or asset, such as the latitude/longitude or chemical name.  We need to separate the timeseries data from these attribute data, and drop the columns not used in the timeseries data.

In [ ]:
df_ts = raw_data[(raw_data.point == 'PRODUCT_VOLUME') | (raw_data.point == 'EST_DAYS_TO_EMPTY')| 
                 (raw_data.point == 'AVERAGE_CHEMICAL_DAILY_USAGE') | (raw_data.point == 'PERCENT_FULL')]

# drop attribute-related columns
del df_ts['latestObservationTimestamp']
del df_ts['latestObservation']

In [ ]:
df_ts.head(20)

Now extract the static attribute data:

In [ ]:
df_attributes = raw_data[(raw_data.point == 'LATITUDE') | (raw_data.point == 'LONGITUDE')| 
                         (raw_data.point == 'TANK_VOLUME') | (raw_data.point == 'CHEMICAL_NAME')]

#Drop timeseries-related columns
del df_attributes['rollupStartTimestampMs']
del df_attributes['rollupEndTimestampMs']
del df_attributes['numericSumValue']
del df_attributes['numericMinValue']
del df_attributes['numericMaxValue']
del df_attributes['numericLastValue']

In [ ]:
df_attributes.head()

In [ ]:
df_attributes.dtypes

Perform some pandas pivot table magic to rearrange the data frame, and ensure the data are the right data type.

In [ ]:
df_attributes = pd.pivot_table(df_attributes, 
                               index=['site','asset'], 
                               values=['latestObservation'],
                               columns=['point'], 
                               aggfunc=lambda x: ' '.join(map(str, x)))
# get rid of multi-level index
df_attributes = df_attributes.xs('latestObservation', axis=1, drop_level=True)
df_attributes = df_attributes.reset_index()

# convert the data with numeric data to the right data type
numeric_cols = ['LATITUDE','LONGITUDE','TANK_VOLUME']
df_attributes[numeric_cols] = df_attributes[numeric_cols].apply(pd.to_numeric)

In [ ]:
df_attributes.head()

Merge timeseries data frame and attribute data frame.  Note that it's an inner join, so we're dropping sites with no timeseries information.

In [ ]:
df = df_ts.merge(df_attributes, how = 'inner', on = ['site','asset'])

In [ ]:
df.head(200)

## EDA (Exploratory Data Analysis)

The primary value appears to be the `PRODUCT_VOLUME`, and the others are derived from that.  Let's look at a few sites timeseries data.

In [ ]:
def plot_sites(sites, df, ycol='numericLastValue'):
    for site in sites:
        df_site = df[df['site']==site]
        for cname, group in df_site.groupby('CHEMICAL_NAME'):
            ax = group.plot(x='rollupEndTimestampMs', y=ycol, title=str(site), marker='+', label=cname)
            ax.set_xlabel('Time')
            ax.set_ylabel('Volume')
            #print(site)

In [ ]:
site_list = list(unique_everseen(df['site']))

In [ ]:
num_sample = 5
variable = 'PERCENT_FULL' #'PRODUCT_VOLUME'

df_variable = df[df['point']==variable].copy()

plot_sites(np.random.choice(site_list, num_sample), df_variable)

Chemicals at most sites follow a logical pattern of steady depletion with occasional fill events rapidly increasing chemical volume levels.  Here are a few with a "typical" pattern:

In [ ]:
normal_sites = [2539767100, 2543205088, 2546199776, 2615146580, 2550863632]
plot_sites(normal_sites, df_variable)

However, there are some timeseries which don't make sense, like the reported volume dropping to zero, or a single value in the timeseries.  These are likely sensor failures of some kind.  Other sites show slow increases in volume, which seems strange but could be from thermal expansion of the chemicals in the tank.

In [ ]:
unusual_sites = [2612863532, 2552756476, 2545207496, 2616812000] #, 2552369860, 2553594584, 2539931224]
plot_sites(unusual_sites, df_variable)

# Empty Tank Prediction 

Normalize time stamp as 'date' to capture all movement within one day and allow for grouping/sorting if needed

In [ ]:
df_variable['date'] = df_variable['rollupEndTimestampMs'].dt.normalize()

In [ ]:
df_variable.sort_values(['site','asset','rollupEndTimestampMs'], inplace=True)

FIXME: this doesn't account for time deltas

In [ ]:
(df_variable['rollupEndTimestampMs'] - df_variable['rollupEndTimestampMs'].shift()).apply(lambda x: x.total_seconds())

In [ ]:
#Solve for daily percent full delta for any give site and asset

df_variable['percent_full_delta'] = np.where(df_variable['site'] == df_variable['site'].shift(), df_variable['numericLastValue'] - df_variable['numericLastValue'].shift() , np.nan)

In [ ]:
def plot_multi(sites, df, ycol=['numericLastValue','percent_full_delta']):
    for site in sites:
        df_site = df[df['site']==site]
        for cname, group in df_site.groupby('CHEMICAL_NAME'):
            ax = group.plot(x='rollupEndTimestampMs', y=ycol, title=str(site), marker='+', label=[cname,'delta'])
            ax.set_xlabel('Time')
            ax.set_ylabel('Volume')

In [ ]:
plot_multi([2546199776,2543205088,2553849160], df_variable) #, ycol='percent_full_delta')

In [ ]:
threshold = 10
df_variable['fill_event'] = df_variable['percent_full_delta'] > threshold

In [ ]:
df_mean_rate_of_depletion = df_variable[df_variable['fill_event']==False].groupby(['site','asset'], as_index=False).agg({"percent_full_delta":"mean"})
df_mean_rate_of_depletion.rename(columns={'percent_full_delta':'mean_rate_of_depletion'}, inplace=True)

In [ ]:
df_mean_rate_of_depletion.head()

***Apply a rate of depletion to the current chemical levels to predict time to empty.***

In [ ]:
#Pull latest percent full levels by site and asset

df_fill_prediction = df_variable.groupby(['site','asset'], as_index=False).agg({"rollupEndTimestampMs":"max"})

In [ ]:
#Ties with depletion rate count, great!

df_fill_prediction.shape

In [ ]:
#Merge lateset time stamp to be predicted on with rates of depletion

df_fill_prediction = pd.merge(df_fill_prediction, df_mean_rate_of_depletion, on=['site','asset'])

In [ ]:
df_fill_prediction.head()

In [ ]:
#Add other available fields at the latest timestamp for each site,asset

df_fill_prediction = pd.merge(df_fill_prediction, df_variable, on=['site','asset','rollupEndTimestampMs'])

In [ ]:
#Data frame clean up

# df_fill_prediction.drop('point', axis=1, inplace=True)
# df_fill_prediction.drop('numericSumValue', axis=1, inplace=True) 
# df_fill_prediction.drop('numericMinValue', axis=1, inplace=True)
# df_fill_prediction.drop('numericMaxValue', axis=1, inplace=True)
# df_fill_prediction.drop('rollupStartTimestampMs', axis=1, inplace=True)
# df_fill_prediction.drop('fill_event', axis=1, inplace=True)
# df_fill_prediction.drop('percent_full_delta', axis=1, inplace=True)

In [ ]:
df_fill_prediction.head()

In [ ]:
#Caclulate predicted days to empty

df_fill_prediction['days_to_empty'] = df_fill_prediction['numericLastValue'] / -(df_fill_prediction['mean_rate_of_depletion'])

In [ ]:
df_fill_prediction.shape

In [ ]:
df_fill_prediction

***Forecast the date at which the tank will reach empty.***

In [ ]:
df_fill_prediction = df_fill_prediction.replace([np.inf, -np.inf], np.nan).dropna(subset=["days_to_empty"], how="any")

For this exercise, drop the pathological rows.  In production, these would be flagged for inspection.

In [ ]:
bad_idx = df_fill_prediction[(df_fill_prediction['mean_rate_of_depletion']>=0) | (df_fill_prediction['mean_rate_of_depletion']<-10)].index
df_fill_prediction.drop(bad_idx, inplace=True)

In [ ]:
df_fill_prediction.shape

In [ ]:
np.sort(df_fill_prediction['mean_rate_of_depletion'])

In [ ]:
df_fill_prediction['mean_rate_of_depletion'].hist(bins=20)

In [ ]:
a=df_fill_prediction.plot(x='mean_rate_of_depletion', y='days_to_empty', linewidth=0, marker='+')
a.set_ylim(0,100)

***Filter for working sensors and prioritize site and chemical fill requirements by time to empty.***

# Clustering (K-Means)

***K-means on location and n = 5 clusters:***
https://www.naftaliharris.com/blog/visualizing-k-means-clustering/

In [ ]:
R = 6371.0 #-- mean radius of curvatureof Earth (km)

In [ ]:
lat0 = np.median(df_fill_prediction['LATITUDE'])
lon0 = np.median(df_fill_prediction['LONGITUDE'])
print(lat0)

In [ ]:
df_fill_prediction['dN'] = np.radians(df_fill_prediction['LATITUDE'] - lat0) * R
df_fill_prediction['dE'] = np.radians(df_fill_prediction['LONGITUDE'] - lon0) * R * np.cos(np.radians(lat0))

In [ ]:
from scipy import stats
from sklearn.cluster import KMeans

#Make a copy of DF
#must_fill = 30  #-- days
#df_tr = df_fill_prediction[df_fill_prediction['days_to_empty'] <= must_fill].copy()
df_tr = df_fill_prediction.copy()

#Standardize
#clmns = ['LONGITUDE', 'LATITUDE']
clmns = ['dE', 'dN']
df_tr_std = stats.zscore(df_tr[clmns])

In [ ]:
#Cluster the data
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df_tr_std)
labels = kmeans.labels_

#Glue back to original data
df_tr['cluster'] = labels

In [ ]:
#Summarize clustering results

df_tr[['site','asset','cluster']].groupby(['cluster']).agg(['count'])

In [ ]:
#Visualize clustering results; scatter plot of LONGITUDE and LATITUDE
sns.lmplot('dE','dN',
           data=df_tr, 
           fit_reg=False, 
           hue="cluster",  
           scatter_kws={"marker": "D", 
                        "s": 100, "alpha": 0.3})
plt.title('Site Location Clusters')
plt.xlabel('Kilometers East')
plt.ylabel('Kilometers North')

***Identify minimum days to empty to prioritize fill schedule by cluster:***

In [ ]:
#Group data by clusters and indentify the minimum days to empty for each cluster

df_cluster_min = df_tr.groupby(['cluster'], as_index=False).agg({"days_to_empty":"min"})

In [ ]:
#Rename the days_to_empty comlumn to prepare merge with df_fill_prediciton

df_cluster_min = df_cluster_min.rename(columns={'days_to_empty':'min_days_to_empty'})

In [ ]:
#Review

df_cluster_min

# Algorithm for "Traveling Salesman Problem" (TSP)

In [ ]:
# Calculate the euclidian distance in n-space of the route r traversing cities (sites) c, ending at the path start.
path_distance = lambda r,c: np.sum([np.linalg.norm(c[r[p]]-c[r[p-1]]) for p in range(len(r))])

# Reverse the order of all elements from element i to element k in array r.
two_opt_swap = lambda r,i,k: np.concatenate((r[0:i],r[k:-len(r)+i-1:-1],r[k+1:len(r)]))

def two_opt(cities,improvement_threshold): # 2-opt Algorithm adapted from https://en.wikipedia.org/wiki/2-opt
    route = np.arange(cities.shape[0]) # Make an array of row numbers corresponding to cities (sites).
    improvement_factor = 1 # Initialize the improvement factor.
    best_distance = path_distance(route,cities) # Calculate the distance of the initial path.
    while improvement_factor > improvement_threshold: # If the route is still improving, keep going!
        distance_to_beat = best_distance # Record the distance at the beginning of the loop.
        for swap_first in range(1,len(route)-2): # From each city (site) except the first and last,
            for swap_last in range(swap_first+1,len(route)): # to each of the cities (sites) following,
                new_route = two_opt_swap(route,swap_first,swap_last) # try reversing the order of these cities (sites)
                new_distance = path_distance(new_route,cities) # and check the total distance with this modification.
                if new_distance < best_distance: # If the path distance is an improvement,
                    route = new_route # make this the accepted best route
                    best_distance = new_distance # and update the distance corresponding to this route.
        improvement_factor = 1 - best_distance/distance_to_beat # Calculate how much the route has improved.
    return route # When the route is no longer improving substantially, stop searching and return the route.

# Source: https://stackoverflow.com/questions/25585401/travelling-salesman-in-scipy

In [ ]:
df_cluster_min.sort_values('min_days_to_empty', inplace=True)
df_cluster_min

In [ ]:
for c in df_cluster_min['cluster'].values:
    df_sites = df_tr[df_tr['cluster']==c]
    sites = df_sites[['dE','dN']].values
    
    # Find a good route with 2-opt ("route" gives the order in which to travel to each site by row number.)
    route = two_opt(sites,0.001)
    
    # Plot the recommended path
    ax = (df_sites.iloc[route]).plot(x='dE',y='dN', marker='D', title='Path for Cluster {}'.format(c))
    ax.set_xlabel('km East')
    ax.set_ylabel('km North')
    ax.get_legend().remove()